In [14]:
#coding=utf-8
import csv
import re  #分词
import pandas as pd

def get_key (dict, value):
    return [k for k, v in dict.items() if v == value]

csv_reader = csv.reader(open('data/match_station.csv', encoding='utf-8'))
Dict = {}
for row in csv_reader:  #0 城区监测点　１　郊区监测点
    Dict[row[0]] = row[1]

In [15]:
#找对应项，构建列表
csv_reader1 = csv.reader(open('data/beijing_17_18_meo.csv', encoding='utf-8'))
#临近气象站点列表
another_factor = {}
for aq in Dict:
    for meo in csv_reader1:
        data_list = []
        if Dict[aq] == meo[0]:
            key = aq + "#" + meo[3]
            for i in range(4, 7):
                data_list.append(meo[i])
            another_factor[key] = data_list

In [24]:
df = pd.read_csv('data/beijing_17_18_aq.csv')
df['temperature'] = 'NaN'
df['pressure'] = 'NaN'
df['humidity'] = 'NaN'
for index in another_factor:
    key = index.split("#")
    print(df[[key[0]][key[1]]])

KeyError: 'aotizhongxin_aq'

In [ ]:
Station_Data = {}  #气象站数据
end_flag = 0
for row in csv_reader1:
    for s0 in station_list:
        single_data = []
        if s0 == row[0]:
            key_list = get_key(Dict, s0)
            for i in range(4, 7):
                single_data.append(row[i])
            for key in key_list:
                new_key = key + "#" + row[3] 
                Station_Data[new_key] = single_data
            break
key_sort_list = sorted(Station_Data.keys())
sorted_Data = {}
for key in key_sort_list:
    sorted_Data[key] = Station_Data[key]
sorted_Data
#print(Station_Data)


In [18]:
out = open('test.csv', 'w', newline='')
csv_writer = csv.writer(out, dialect='excel')
for key in sorted_Data:
    key_list = []
    key_list = key.split('#')
    for i in sorted_Data[key]:
        key_list.append(i)
    csv_writer.writerow(key_list)
print("write over")
    

write over


In [23]:
import pandas as pd
df = pd.read_csv('data/beijing_17_18_meo.csv')
df.utc_timec_time

0         2017-01-30 16:00:00
1         2017-01-30 17:00:00
2         2017-01-30 18:00:00
3         2017-01-30 19:00:00
4         2017-01-30 20:00:00
5         2017-01-30 21:00:00
6         2017-01-30 22:00:00
7         2017-01-30 23:00:00
8         2017-01-31 00:00:00
9         2017-01-31 01:00:00
10        2017-01-31 02:00:00
11        2017-01-31 03:00:00
12        2017-01-31 04:00:00
13        2017-01-31 05:00:00
14        2017-01-31 06:00:00
15        2017-01-31 07:00:00
16        2017-01-31 08:00:00
17        2017-01-31 09:00:00
18        2017-01-31 10:00:00
19        2017-01-31 11:00:00
20        2017-01-31 12:00:00
21        2017-01-31 13:00:00
22        2017-01-31 14:00:00
23        2017-01-31 15:00:00
24        2017-01-31 16:00:00
25        2017-01-31 17:00:00
26        2017-01-31 18:00:00
27        2017-01-31 19:00:00
28        2017-01-31 20:00:00
29        2017-01-31 21:00:00
                 ...         
158017    2018-01-30 10:00:00
158018    2018-01-30 11:00:00
158019    

In [24]:
cd ..

/media/yu/Study/lesson_file/Junior/Summer-Term/Data-Mining/Homework


In [25]:
ls

Git-Hub/  HW1/  HW2/  HW3/  Hw4/  kdd/  kdd-cup/


In [26]:
cd kdd

/media/yu/Study/lesson_file/Junior/Summer-Term/Data-Mining/Homework/kdd


In [27]:
ls


api_submit.py*   DataSet.py*      forest.ipynb*        skdemo.py*
data/            Distance.ipynb*  map_function.ipynb*  test.csv*
datapipe.ipynb*  Distance.py*     __pycache__/


In [38]:
test = pd.read_csv('test.csv')
test.index = test['2017-01-01 00:00:00']
test = test.loc[df.utc_time[:20]]
import numpy as np
test.index = np.arange(len(test))
test

,aotizhongxin_aq,2017-01-01 00:00:00,-5.96,1019.95,68.6
0,aotizhongxin_aq,2017-01-30 16:00:00,-5.89,1026.03,14.58
1,beibuxinqu_aq,2017-01-30 16:00:00,-7.41,1004.66,14.36
2,dongsi_aq,2017-01-30 16:00:00,-5.56,1027.10,14.71
3,fengtaihuayuan_aq,2017-01-30 16:00:00,-5.57,1024.94,14.21
4,guanyuan_aq,2017-01-30 16:00:00,-5.57,1024.94,14.21
5,gucheng_aq,2017-01-30 16:00:00,-5.85,1015.61,13.93
6,nongzhanguan_aq,2017-01-30 16:00:00,-5.80,1030.18,15.10
7,tiantan_aq,2017-01-30 16:00:00,-5.89,1026.03,14.58
8,wanliu_aq,2017-01-30 16:00:00,-5.98,1021.89,14.07
9,wanshouxigong_aq,2017-01-30 16:00:00,-5.56,1027.10,14.71


In [35]:
test.loc['2017-01-01 01:00:00']

,aotizhongxin_aq,2017-01-01 00:00:00,-5.96,1019.95,68.6
2017-01-01 00:00:00,,,,,
2017-01-01 01:00:00,aotizhongxin_aq,2017-01-01 01:00:00,-3.36,1019.82,58.43
2017-01-01 01:00:00,beibuxinqu_aq,2017-01-01 01:00:00,-3.19,998.94,60.26
2017-01-01 01:00:00,dongsi_aq,2017-01-01 01:00:00,-3.36,1020.76,60.87
2017-01-01 01:00:00,fengtaihuayuan_aq,2017-01-01 01:00:00,-3.04,1018.64,59.58
2017-01-01 01:00:00,guanyuan_aq,2017-01-01 01:00:00,-3.04,1018.64,59.58
2017-01-01 01:00:00,gucheng_aq,2017-01-01 01:00:00,-3.01,1009.55,59.70
2017-01-01 01:00:00,nongzhanguan_aq,2017-01-01 01:00:00,-3.62,1023.86,58.87
2017-01-01 01:00:00,tiantan_aq,2017-01-01 01:00:00,-3.36,1019.82,58.43
2017-01-01 01:00:00,wanliu_aq,2017-01-01 01:00:00,-3.11,1015.78,58.00
